In [ ]:
# -*- coding: utf-8 -*-

# Standard library imports
import os
import random
from PIL import Image, ImageEnhance

# Third-party imports
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from albumentations import (
    HueSaturationValue, RGBShift, Blur, RandomRotate90, RandomFog, 
    RandomRain, RandomSnow, Spatter, GaussNoise, CLAHE, CoarseDropout, 
    MedianBlur, GaussianBlur, PixelDropout
)
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, Callback
from tensorflow.keras.metrics import (
    AUC, Precision, Recall, TruePositives, 
    TrueNegatives, FalsePositives, FalseNegatives
)
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B2

In [ ]:
# Augmentation definitions using Albumentations library
# Each augmentation is described briefly for better understanding. 

# 1. Randomly rotate the image by 90 degrees
rotate_aug = RandomRotate90(
    always_apply=True, 
    p=1.0
)

# 2. Adjust hue, saturation, and value with specified limits
hue_sat_val = HueSaturationValue(
    always_apply=False, 
    p=1.0, 
    hue_shift_limit=(0, 0), 
    sat_shift_limit=(-5, 20), 
    val_shift_limit=(-5, 10)
)

# 3. Apply a blur effect with a specified blur limit
blur_aug = Blur(
    blur_limit=(3, 7), 
    p=1.0
)

# 4. Add fog with adjustable coefficients
fog = RandomFog(
    fog_coef_lower=0.1, 
    fog_coef_upper=0.4, 
    alpha_coef=0.5, 
    always_apply=False, 
    p=1.0
)

# 5. Simulate rain with configurable properties
random_rain = RandomRain(
    always_apply=False, 
    p=1.0, 
    slant_lower=-5, 
    slant_upper=5, 
    drop_length=20, 
    drop_width=1, 
    drop_color=(0, 0, 0), 
    blur_value=3, 
    brightness_coefficient=1.0, 
    rain_type='drizzle'
)

# 6. Simulate snowfall with adjustable intensity and brightness
random_snow = RandomSnow(
    always_apply=False, 
    p=1.0, 
    snow_point_lower=0.1, 
    snow_point_upper=0.25, 
    brightness_coeff=1.2
)

# 7. Apply spatter effects, useful for simulating dirt or water droplets
splatter = Spatter(
    always_apply=False, 
    p=1.0, 
    mean=(0.46, 0.46), 
    std=(2.3, 2.3), 
    gauss_sigma=(0.99, 0.99), 
    intensity=(0.2, 0.2), 
    cutout_threshold=(-0.39, -0.39), 
    mode=['rain']
)

# 8. Add Gaussian noise to the image
gaus_noise = GaussNoise(
    always_apply=False, 
    p=1.0, 
    var_limit=(10.0, 100.0), 
    per_channel=True, 
    mean=0.0
)

# 9. Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for better contrast
clahe = CLAHE(
    always_apply=False, 
    p=1.0, 
    clip_limit=(1, 5), 
    tile_grid_size=(10, 10)
)

# 10. Randomly drop regions of the image (CoarseDropout)
coarse_dropout = CoarseDropout(
    always_apply=False, 
    p=1.0, 
    max_holes=30, 
    max_height=10, 
    max_width=10, 
    min_holes=10, 
    min_height=8, 
    min_width=8, 
    fill_value=(0, 0, 0), 
    mask_fill_value=None
)

# 11. Apply median blur with a specified blur limit
median_blur = MedianBlur(
    always_apply=False, 
    p=1.0, 
    blur_limit=(3, 7)
)

# 12. Apply Gaussian blur with adjustable sigma and blur limits
gaussian_blur = GaussianBlur(
    always_apply=False, 
    p=1.0, 
    blur_limit=(3, 7), 
    sigma_limit=(0.0, 0.0)
)

# 13. Randomly drop pixels (PixelDropout) for regularization
pixel_dropout = PixelDropout(
    always_apply=False, 
    p=1.0, 
    dropout_prob=0.05, 
    per_channel=1, 
    drop_value=(0, 0, 0), 
    mask_drop_value=None
)




In [ ]:
def preprocess_input_augment(x0):
    """
    Applies random augmentations to the input image after being processed 
    by the Keras ImageDataGenerator.

    Args:
        x0 (numpy.ndarray): The input image array.

    Returns:
        numpy.ndarray: The augmented image.
    """
    # Convert the image to uint8 format
    x0 = np.uint8(x0)

    # Apply a random 90-degree rotation
    x0 = rotate_aug(image=x0)['image']

    # Generate a random number to determine augmentation
    randomnr = random.randrange(18)

    # Apply augmentations based on the random number
    if randomnr in [0, 1, 3]:  # No augmentation
        pass
    elif randomnr == 4:  # Increase color saturation (RGB)
        image = Image.fromarray(x0)
        enhance_filter = ImageEnhance.Color(image)
        enhanced_image = enhance_filter.enhance(2)  # Factor of 2 for saturation
        x0 = np.asarray(enhanced_image, dtype=np.uint8)
    elif randomnr == 5:  # Adjust hue, saturation, and value
        x0 = hue_sat_val(image=x0)['image']
    elif randomnr == 6:  # Apply blur
        x0 = blur_aug(image=x0)['image']
    elif randomnr == 7:  # Add fog
        x0 = fog(image=x0)['image']
    elif randomnr == 8:  # Simulate rain
        x0 = random_rain(image=x0)['image']
    elif randomnr == 9:  # Simulate snow
        x0 = random_snow(image=x0)['image']
    elif randomnr == 10:  # Apply spatter effect
        x0 = splatter(image=x0)['image']
    elif randomnr == 11:  # Add Gaussian noise
        x0 = gaus_noise(image=x0)['image']
    elif randomnr == 12:  # Apply CLAHE (contrast enhancement)
        x0 = clahe(image=x0)['image']
    elif randomnr == 13:  # Apply coarse dropout
        x0 = coarse_dropout(image=x0)['image']
    elif randomnr == 14:  # Apply CLAHE again (redundant but kept)
        x0 = clahe(image=x0)['image']
    elif randomnr == 15:  # Apply median blur
        x0 = median_blur(image=x0)['image']
    elif randomnr == 16:  # Apply Gaussian blur
        x0 = gaussian_blur(image=x0)['image']
    elif randomnr == 17:  # Apply pixel dropout
        x0 = pixel_dropout(image=x0)['image']

    return x0


def save(model, prefix):
    """
    Saves the model weights and architecture to files.

    Args:
        model (keras.Model): The model to save.
        prefix (str): The file prefix for saving the model.
    """
    # Save the model weights
    model.save_weights(f"{prefix}.h5")

    # Serialize the model architecture to JSON
    model_json = model.to_json()
    with open(f"{prefix}.json", "w") as json_file:
        json_file.write(model_json)
        
def get_folder_count(folder):
    file_count = sum(len(files) for _, _, files in os.walk(folder))
    return file_count

In [ ]:
# Configuration Settings
batch_size = 32  # Number of images per batch. Adjusted for high resolution (384x384px).
nr_epochs = 1000  # Total number of training epochs
img_width, img_height = 384, 384  # Image dimensions
input_shape = (img_width, img_height, 3)  # Input shape for the model

# Model and Dataset Information
model_name = 'effinet_v1'  # Name of the model
load_imagenet_weights = False  # Flag to determine if ImageNet weights should be loaded

# Dataset Paths
train_data_dir = '/trainset'  # Directory for training data
validation_data_dir = '/valset'  # Directory for validation data

# Retrieve Dataset Information
nr_train_samples = get_folder_count(train_data_dir)
nr_val_samples = get_folder_count(validation_data_dir)

# Print Dataset Details
print("Loading dataset:")
print(f"Number of training samples: {nr_train_samples}")
print(f"Number of validation samples: {nr_val_samples}")

# Class Information
nr_classes = 2  # Number of classes
tags = ['positive', 'negative']  # Class labels

# Set TensorFlow Backend Image Data Format
K.set_image_data_format('channels_last')

In [ ]:
# Data Generators for Training and Validation

# Training Data Generator with Augmentations
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_augment,  # Custom preprocessing function
    brightness_range=[0.97, 1.03],  # Adjust brightness within the specified range
    horizontal_flip=True,  # Random horizontal flip
    vertical_flip=True  # Random vertical flip
)

# Validation/Test Data Generator without Augmentations
test_datagen = ImageDataGenerator()

# Training Data Generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Path to training data
    shuffle=True,  # Shuffle training data
    target_size=(img_width, img_height),  # Resize images to specified dimensions
    batch_size=batch_size,  # Batch size
    class_mode='binary'  # Binary classification
)

# Validation Data Generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,  # Path to validation data
    shuffle=True,  # Shuffle validation data
    target_size=(img_width, img_height),  # Resize images to specified dimensions
    batch_size=batch_size,  # Batch size
    class_mode='binary'  # Binary classification
)

In [ ]:
def build_model(input_shape, nb_classes=2):
    """
    Builds a model with EfficientNetV2B2 as the base and custom head.
    
    Args:
        input_shape (tuple): Shape of the input image.
        nb_classes (int): Number of output classes. Default is 2.
    
    Returns:
        model (keras.Model): Compiled model.
    """
    # Initialize the base model
    base_model = EfficientNetV2B2(weights=None, input_shape=input_shape, include_top=False)
    
    # Set base model layers as trainable
    base_model.trainable = True

    # Create the head of the model
    head_model = base_model.output
    head_model = GlobalAveragePooling2D()(head_model)
    head_model = Dropout(0.2)(head_model)
    output_layer = Dense(nb_classes-1, activation="sigmoid")(head_model)

    # Build the final model
    model = Model(inputs=base_model.input, outputs=output_layer)

    return model

In [ ]:
# Load EfficientNet model
print("Loading Efficient model")
model = build_model(input_shape, nb_classes=2)

# Define optimizer with configuration
optimizer = SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile the model with loss function and metrics
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        "binary_accuracy", 
        AUC(),
        Precision(),
        Recall(),
        TruePositives(),
        TrueNegatives(),
        FalsePositives(),
        FalseNegatives()
    ]
)

In [ ]:
# Define callbacks
print("Setting up callbacks...")

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="effinet_epoch.{epoch:02d}.h5",
    save_weights_only=True,
    save_freq="epoch",
    save_best_only=False
)

csv_logger = CSVLogger(f"{model_name}_model_history_log.csv", append=True)

callbacks = [model_checkpoint_callback, csv_logger]

# Start training
print("Starting model training...")

history = model.fit(
    x=train_generator,
    epochs=nr_epochs,
    validation_data=validation_generator,
    steps_per_epoch=nr_train_samples // batch_size,
    validation_steps=nr_val_samples // batch_size,
    callbacks=callbacks,
    workers=8,
    max_queue_size=16,
    use_multiprocessing=True
)

# Save the trained model
save(model, model_name)